In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs

In [ ]:
%%time
a,l = load_rfam(full= True, add_cov= False)

In [ ]:
%%time
def atograph(ali):
    graph = ali2graph.set_weight_label(ali)
    graph = ali2graph.dillute(graph)
    graph = ali2graph.set_weight(graph)
    graph = ali2graph.donest(graph)
    return graph

graphss = ut.xxmap(atograph, a)
matrix = ygraphs.vectorize_graphs(graphss, normalization=True,
                      min_r = 2,min_d=1,
                      inner_normalization=True)

# sml.knn_accuracy(matrix,l)

In [ ]:
# dist = 1- (matrix @ matrix.T).toarray()
# dist[dist<0] =0

from kiez import Kiez
# k_inst = Kiez(algorithm='SklearnNN', hubness='csls', n_candidates = 100,  algorithm_kwargs= {'metric' : 'precomputed'})
k_inst = Kiez(algorithm='SklearnNN', hubness='csls', n_candidates = 100,  algorithm_kwargs= {'metric' : 'cosine'})
k_inst.fit(matrix.toarray())
dist, neigh_ind = k_inst.kneighbors()
        
# return sml.average_precision_limited(dist,neigh_ind,labels)


In [ ]:
dist.shape

In [ ]:
%%time
from  sklearn.neighbors import KNeighborsClassifier



yy= np.array(l)
indices = neigh_ind
neighbor_labels = yy[indices]
neighbor_labels = neighbor_labels[:, 1:]
def searchrank(i,l):
    row = neighbor_labels[i] 
    indices = np.where(row == l)[0]
    return indices[0] if len(indices)>0 else 999999
ranks = np.array([searchrank(i,lab) for i,lab in enumerate(yy) if lab != 0])
ranks

In [ ]:
ranks.shape

In [ ]:
%%time
def calcranks(matrix,l):
    knn = KNeighborsClassifier(n_neighbors=100)
    yy= np.array(l)
    knn.fit(matrix,yy)
    _, indices = knn.kneighbors(matrix)
    neighbor_labels = yy[indices]
    neighbor_labels = neighbor_labels[:, 1:]
    def searchrank(i,l):
        row = neighbor_labels[i] 
        indices = np.where(row == l)[0]
        return indices[0] if len(indices)>0 else 999999
    ranks = [searchrank(i,lab) for i,lab in enumerate(yy) if lab != 0]
    return np.array(ranks)
a2,l2 = load_rfam(full= True, add_cov= False)
mat = ygraphs.vectorize_alignments(a, min_r = 2, min_d=1)
ranks2 = calcranks(mat,l2)

In [ ]:
# PLOT importance weighted vectorization  vs normal vectorization , do we have at least 1 NN in the closest X instances?
plt.plot([sum(ranks < x)/sum(l != 0) for x in range(1,11)])
plt.plot([sum(ranks2 < x)/sum(l2 != 0) for x in range(1,11)])

In [ ]:
plt.plot([sum(ranks < x)/sum(l != 0) for x in range(100)])
plt.plot([sum(ranks2 < x)/sum(l2 != 0) for x in range(100)])

In [ ]:
# are there different successrates for more populated families? 
manyseq, fewseq = filter_by_seqcount(a)

def pl(x):
    x.append(0)
    mask = [ ll in x for ll in l ]
    ranks = calcranks(matrix[mask], l[mask])
    # ranks = calcranks(mat[mask], l[mask])
    data = [sum(ranks < xx)/sum(l[mask] != 0) for xx in range(1,21)]
    plt.plot(data)
    return data

dg = pl(manyseq)
db = pl(fewseq)

In [ ]:
todi = lambda x,y: [{'clan hit':e, 'neighbors':i, 'experiment':y} for i,e in enumerate(x)]
import pandas as pd
import seaborn as sns
g,b = ([0.26717557251908397,
  0.2824427480916031,
  0.29770992366412213,
  0.32061068702290074,
  0.33587786259541985,
  0.3511450381679389,
  0.35877862595419846,
  0.3816793893129771,
  0.3816793893129771,
  0.3969465648854962,
  0.40458015267175573,
  0.40458015267175573,
  0.4122137404580153,
  0.4122137404580153,
  0.4122137404580153,
  0.4122137404580153,
  0.4198473282442748,
  0.42748091603053434,
  0.4351145038167939,
  0.45038167938931295],
 [0.3231939163498099,
  0.3726235741444867,
  0.41064638783269963,
  0.43346007604562736,
  0.44106463878326996,
  0.4600760456273764,
  0.4714828897338403,
  0.4790874524714829,
  0.49049429657794674,
  0.49429657794676807,
  0.5057034220532319,
  0.5171102661596958,
  0.5209125475285171,
  0.5285171102661597,
  0.5285171102661597,
  0.5285171102661597,
  0.5285171102661597,
  0.532319391634981,
  0.532319391634981,
  0.5361216730038023])
data = pd.DataFrame(todi(dg,'manySequences, boost_flank')+
                    todi(db,'fewSequences, boost_flank')+
                    todi(g,'manyseq, no boost')+
                    todi(b,'fewseq, no boost'))
# sns.lineplot(x="neighbors", y="clan hit", hue="experiment",  data=data)

data = pd.DataFrame(todi(dg,'manySequences')+
                    todi(db,'fewSequences'))

sns.lineplot(x="neighbors", y="clan hit",
             hue="experiment", 
             data=data)

In [ ]:
#ranks = calcranks(matrix, l)

In [ ]:
ranks

In [ ]:
# if we know the neighbor, can we find the whole familiy? 

if False:
    knn = KNeighborsClassifier(n_neighbors=100)
    yy= np.array(l)
    knn.fit(matrix,yy)
    _, indices = knn.kneighbors(matrix[yy!=0])


# are two enough to find the rest? 

In [ ]:
cm_dict_ab={}
cm_dict_ba={}
with open('rfam-weakpairs','r') as f:
    for line in f:
        if line:
            sline = line.split(' ')
            cm_dict_ab[sline[0]+sline[1]] = line
            cm_dict_ba[sline[1]+sline[0]] = line
            

In [ ]:
vec = eg.Vectorizer( min_r = 2,min_d=1)
import structout.intlistV2 as so 

def nice(values, colorids=[]):
    symbols = '▁▂▃▄▅▆▇█'
    values = np.array(values)
    values = so.binning(values, count = len(symbols), ylim=False)
    color = lambda x: '\x1b[1;3%d;48m%s\x1b[0m' % (1, x)
    if not colorids: 
        return ''.join([symbols[v] for v in values])
    else:
        return ''.join([symbols[v] if i not in colorids else color(symbols[v]) for i,v in enumerate(values)])


r = []
done = set()
needles = []
rowz = []
for i,ind in enumerate(neigh_ind):
    # we want r0 and r1 to be equal and >0.. 
    rowlabels = yy[ind]
    if rowlabels[0] == 0 or rowlabels[0]!=rowlabels[1]:
       continue 
    # we want 
    c,d = ind[0], ind[1]
    if (c,d) in done:
        continue
    else:
        done.add((d,c))
    
    needle = np.array([ (a + b) /2  for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0])])
    needles.append(needle)
    rowz.append((i,c,d))
    #input = np.log1p(matrix[[c,d],:])
    #needle = np.asarray(np.sum(input, axis = 0))[0]
    #input = matrix[[c,d],:]
    #needle = np.asarray(np.sum(input, axis = 0))[0]
    # FAB
    #index_helper = np.array([((a+b)/2) if a and b else 0 for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0]) ])
    # needle = np.array([(a+b)/2 for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0]) if a and b])
    #indices = np.argsort(-np.dot(matrix.todense()[:, index_helper > 0] , needle.T )).A1 [:50]
    
    # dist, indices = k_inst.kneighbors([needle])
    
k_inst = Kiez(algorithm='SklearnNN', hubness='csls',  n_candidates = 100,  
              algorithm_kwargs= {'metric' : 'cosine'})
k_inst.fit(np.array(needles),matrix.toarray() )
dist, indices2 = k_inst.kneighbors()

for indices,rid in zip( indices2, rowz):
    i,c,d = rid
    # CLASS IDS 
    r.append(str(yy[indices]))
    print(r[-1])
    # RFAM IDS 
    # IMPORTANCES
    for i in indices[:10]: # FIST 20 HITS 
        graph = graphss[i]
        seqid_scr = {}
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            seqid_scr[n]= (ve*needle)[0]
        score_per_nucleotide = [ seqid_scr.get(j,0) for j in range(a[i].alignment.shape[1])]


        conserved = [ nid for nid,di in graphss[i].nodes(data=True) if di['weight']> .5]
        #### print per entry... 
        # NAME,  score 
        # position score + highlight 
        # structure line 
        # 5 ali lines 
        # newline 
        r.append(f"{a[i].gf['DE']} {a[i].gf['ID']}") # ! score later 
        # print(r[-1])
        r.append(f"{nice(score_per_nucleotide, conserved)}") 
        # print(r[-1])
        r.append(f"{a[i].gc['SS_cons']}") 
        # print(r[-1])
        for row in a[i].alignment[:5]:
            r.append(f'{"".join(row)}') 
        # print(r[-1])

        labels = a[c].label, a[i].label
        k = '%s.cm%s.cm' % labels
        r.append(cm_dict_ab.get(k,'None'))
        r.append(cm_dict_ba.get(k,'None'))
        labels = a[d].label, a[i].label
        k = '%s.cm%s.cm' % labels
        r.append(cm_dict_ab.get(k,'None'))
        r.append(cm_dict_ba.get(k,'None'))
        r.append(f'') 
        
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle
        return graph
    #gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 
    def draw(g):
        if len(g) < 400: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    # [draw(g) for g in gra]




In [ ]:
r

In [ ]:
np.array(needles).shape

In [ ]:
matrix.toarray().shape

In [ ]:
r[2]

In [ ]:
r = [e if e is not None else "None" for e in r]

In [ ]:
with open('data.txt','w') as file:
    file.write('\n'.join(r))
!cat data.txt | aha > data.html

In [ ]:
import sklearn
sklearn.__file__

In [ ]:
# CAN WE FIND NEW INSTANCES OF A CLUSTER IF WE PROVIDE THE WHOLE CLUSTER?

knn = KNeighborsClassifier(n_neighbors=100)
yy= np.array(l)
knn.fit(matrix ,yy)

y_needle =  {}

for y in np.unique(yy): 
    if y > 0:
        # pairwise search ... 
        indices = np.where (yy==y)[0]
        for a,b in [(i,j) for enu,i in enumerate(indices) for j in indices[enu+1:]]:
            input = np.log1p(matrix[[c,d],:])
            needle = np.asarray(np.sum(input, axis = 0))[0]           
        
    
        
        # input = matrix[yy == y]
        # needle = np.asarray(np.all((input > 0).todense(), axis = 0))
        
        dist, indices = knn.kneighbors(needle)
        print(y)
        print(yy[indices])
        print(dist)
        y_needle[y] = (needle, indices) 


In [ ]:
def show(y):
    needle, indices = y_needle[y]
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle[0]
        return graph
    gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 
    
    def draw(g):
        if len(g) < 300: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    [draw(g) for g in gra]


show(8)


In [ ]:
>>> rng = np.random.RandomState(0)
>>> source = rng.rand(61,65537)
>>> target = rng.rand(4108,65537)


# Step 2: Create a Kiez object
kiezz = Kiez(hubness='localscaling')  # 'ls' stands for Local Scaling, you can also use 'mp' for Mutual Proximity

# Step 3: Fit the Kiez object to your data
kiezz.fit(source)

# Step 4: Transform the data to get hubness-corrected distances
X_corrected = kiez.transform(source)

# Step 5: Convert the corrected data to a distance matrix
#distance_matrix = nn.kneighbors_graph(X_corrected, mode='distance').toarray()

In [ ]:
nn_ind.shap